<a href="https://colab.research.google.com/github/kartavya1031/Book_recomendation/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Recommendation_System/Books.csv', sep = ",", on_bad_lines='skip', encoding='latin-1')

<ipython-input-44-647c610214f4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/content/drive/MyDrive/Recommendation_System/Books.csv', sep = ",", on_bad_lines='skip', encoding='latin-1')


In [ ]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [ ]:
books = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]

In [ ]:
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'}, inplace=True)

In [ ]:
books.head(2)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [ ]:
users = pd.read_csv('/content/drive/MyDrive/Recommendation_System/Users.csv', sep = ",", on_bad_lines='skip', encoding='latin-1')

In [ ]:
users.head(2)
users.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'}, inplace=True)

In [ ]:
users.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/Recommendation_System/Ratings.csv', sep = ",", on_bad_lines='skip', encoding='latin-1')

In [ ]:
ratings.head(2)
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'}, inplace=True)

In [ ]:
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [ ]:
books.shape

(271360, 5)

In [ ]:
users.shape

(278858, 3)

In [ ]:
ratings.shape

(1149780, 3)

In [ ]:
x=ratings['user_id'].value_counts()>200

In [ ]:
y=x[x].index

In [ ]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [ ]:
ratings = ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings.shape

(526356, 3)

In [ ]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [ ]:
ratings_with_books = ratings.merge(books, on = 'ISBN')

In [ ]:
ratings_with_books.shape

(487671, 7)

In [ ]:
number_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
number_rating.rename(columns={'rating':'number_of_ratings'}, inplace=True)

In [ ]:
number_rating

,title,number_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
160264,Ã?Ã?ber die Pflicht zum Ungehorsam gegen den...,3
160265,Ã?Ã?lpiraten.,1
160266,Ã?Ã?rger mit Produkt X. Roman.,1
160267,Ã?Ã?stlich der Berge.,1


In [ ]:
final_rating = ratings_with_books.merge(number_rating, on = 'title')

In [ ]:
final_rating.shape

(487671, 8)

In [ ]:
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]

In [ ]:
final_rating.shape

(61853, 8)

In [ ]:
final_rating.drop_duplicates(['user_id','title'], inplace = True)

<ipython-input-93-6d9ab256ad29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['user_id','title'], inplace = True)


In [ ]:
final_rating.shape

(59850, 8)

In [ ]:
book_pivot = final_rating.pivot_table(columns ='user_id',index = 'title', values= 'rating')

In [ ]:
book_pivot.fillna(0,inplace = True)

In [ ]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [ ]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')

In [ ]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[54,:].values.reshape(1,-1), n_neighbors=6)

In [ ]:
suggestions

array([[237, 238, 240, 241, 184, 536]])

In [ ]:
distances

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Animal Farm', 'Exclusive', 'Jacob Have I Loved', 'Second Nature',
       'Pleading Guilty', 'No Safe Place'],
      dtype='object', name='title')


In [ ]:
book_pivot.index[54]

'Animal Farm'

In [ ]:
np.where(book_pivot.index=='Animal Farm')[0][0]

54

In [ ]:
def recomend_book(book_name):
  book_id = np.where(book_pivot.index==book_name)[0][0]
  distances, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6)

  for i in range(len(suggestions)):
    if i == 0:
      print('The suggestions for', book_name, 'are:')
    if not i:
      print(book_pivot.index[suggestions[i]])

In [ ]:
recomend_book('Second Nature')

The suggestions for Second Nature are:
Index(['Second Nature', 'Exclusive', 'Jacob Have I Loved', 'No Safe Place',
       'The Cradle Will Fall', 'Long After Midnight'],
      dtype='object', name='title')
